# Recommendation System

## User-Based Collaborative Filtering

## Notebook by [WenyiXu](https://github.com/xuwenyihust)

### Loading the data

In [26]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats.stats import pearsonr

from sklearn import cross_validation

fulldf=pd.read_csv("https://raw.githubusercontent.com/xuwenyihust/Recommendation-Systems/master/data/amazon-fine-foods.csv")
fulldf = fulldf[:20000]

C:\Users\Wenyi Xu\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (1,2,3,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [27]:
print(fulldf.head(5))
print(fulldf.shape)

    Id   ProductId          UserId                      ProfileName  \
0  1.0  B001E4KFG0  A3SGXH7AUHU8GW                       delmartian   
1  2.0  B00813GRG4  A1D87F6ZCVE5NK                           dll pa   
2  3.0  B000LQOCH0   ABXLMWJIXXAIN  Natalia Corres "Natalia Corres"   
3  4.0  B000UA0QIQ  A395BORC6FGVXV                             Karl   
4  5.0  B006K2ZZ7K  A1UQRSCLF8GW1T    Michael D. Bigham "M. Wassir"   

   HelpfulnessNumerator  HelpfulnessDenominator  Score          Time  \
0                   1.0                     1.0    5.0  1.303862e+09   
1                   0.0                     0.0    1.0  1.346976e+09   
2                   1.0                     1.0    4.0  1.219018e+09   
3                   3.0                     3.0    2.0  1.307923e+09   
4                   0.0                     0.0    5.0  1.350778e+09   

                 Summary                                               Text  
0  Good Quality Dog Food  I have bought several of the Vitalit

### Train/Test Split

Split the train & test dataset, use the train subset to train the model, and validate the model using test subset.

In [28]:
X = fulldf[['Id', 'ProductId', 'UserId']]
y = fulldf[['Score']]

X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

print(X_train.head(5))
print(X_test.head(5))

(16000, 3) (16000, 1)
(4000, 3) (4000, 1)
          Id   ProductId          UserId
5894  5895.0  B003KLSZGW  A1DXTSHQEVQJTR
3728  3729.0  B004SFNFEG  A3SIZC600PA0AW
8958  8959.0  B0007A0AP8  A1OI15XSWJ8FFX
7671  7672.0  B001E5DZTS  A1JY70V80B1F78
5999  6000.0  B003SE52K8   AJNSMUB2HU1M8
            Id   ProductId          UserId
10650  10651.0  B001KVNDM0  A3C56IXBT4ETXA
2041    2042.0  B001E5E29A   AQMBGG9S7964D
8668    8669.0  B003VXFK44  A3SFS8R1T6JJM9
1114    1115.0  B003UDV9SG  A2W09PJ1TTHQ6H
13902  13903.0  B002KE92UI  A2QCBK7UKFTEOC


### User-item matrix construction

Transform the original table into a user-item matrix

In [29]:
traindf = X_train
traindf.loc[:,'Score'] = y_train
print(traindf.head(5))
matrix = traindf.pivot_table(index='UserId', columns='ProductId', values='Score')
print(matrix.shape)
print(matrix.head(5))

C:\Users\Wenyi Xu\Anaconda3\lib\site-packages\pandas\core\indexing.py:284: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Wenyi Xu\Anaconda3\lib\site-packages\pandas\core\indexing.py:461: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


          Id   ProductId          UserId  Score
5894  5895.0  B003KLSZGW  A1DXTSHQEVQJTR    5.0
3728  3729.0  B004SFNFEG  A3SIZC600PA0AW    5.0
8958  8959.0  B0007A0AP8  A1OI15XSWJ8FFX    5.0
7671  7672.0  B001E5DZTS  A1JY70V80B1F78    5.0
5999  6000.0  B003SE52K8   AJNSMUB2HU1M8    5.0
(14425, 2405)
ProductId       B00002NCJC  B00002Z754  B00005V3DC  B000084DVR  B000084E1U  \
UserId                                                                       
A1001TYW5FZYD9         NaN         NaN         NaN         NaN         NaN   
A1003ACBU13X6E         NaN         NaN         NaN         NaN         NaN   
A1004703RC79J9         NaN         NaN         NaN         NaN         NaN   
A10080F3BO83XV         NaN         NaN         NaN         NaN         NaN   
A1011I2PCI4K8B         NaN         NaN         NaN         NaN         NaN   

ProductId       B000084EK4  B000084EK5  B000084EK6  B000084EK7  B000084EK8  \
UserId                                                                   

### Brief Exploratory analysis

In [30]:
#item_review_counts = [0 for x in range(matrix.shape[0])]
#user_review_count = [0 for x in range(matrix.shape[1])]


### User-to-user similarity

Define a function to compute the similarities between users.

Based on Pearson correlation.

In [38]:
def com_sim(user0_reviews, user1_reviews):
    return pearsonr(user0_reviews, user1_reviews)[0] 

### Get items that a user has reviewed

In [40]:
def items_reviewed(user):
    return traindf[traindf['UserId'] == user]['ProductId'].unique()

### Reviews for common items

In [68]:
def common_items(user0, user1):
    user0_items = items_reviewed(user0)
    user1_items = items_reviewed(user1)
    return set(user0_items).intersection(user1_items)

### Get reviews of a list of items rated by a given user

In [73]:
def get_reviews(user, set_of_items):
    mask = (traindf['ProductId'].isin(set_of_items)) & (traindf['UserId']==user)
    reviews = traindf[mask]
    reviews = reviews[reviews.UserId.duplicated()==False]
    return reviews

### K nearest users

Given an active user and an item that hasn't been rated by him/her, find the K nearest users to that active one that have rated the item.

In [71]:
def k_nearest(user0, k):
    all_users = fulldf['UserId'].unique().tolist()
    knu = []
    for user1 in all_users:
        if user0 != user1:
            both_items = common_items(user0, user1)
            reviews_user0 = get_reviews(user0, both_items)
            reviews_user1 = get_reviews(user1, both_items)
            if len(both_items) == 0:
                sim = 0
            else:
                sim = com_sim(reviews_user0, reviews_user1)
            if len(knu) < k:
                knu.append([user1, sim])
                knu = sorted(knu, key=lambda similarity: similarity[1])
            else:
                if sim > knu[-1][1]:
                    knu[-1] = [user1, sim]
                    knu = sorted(knu, key=lambda similarity: similarity[1])
    return knu

Test the K nearest users function

In [74]:
#all_users = fulldf['UserId'].unique().tolist()
#print(all_users[0])
#print(type(all_users[0]))

#print(set)
print(k_nearest('A1001TYW5FZYD9', 1))

TypeError: unsupported operand type(s) for +: 'float' and 'str'